In [1]:
import os
import torch, torchvision

os.chdir("/home/kishen/documents/python_projects/stanford_dogs/")
from src.data import ImageDataset

In [2]:
#read in image data
path = "/home/kishen/documents/python_projects/stanford_dogs/data/Images"

transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Resize((224, 224)),
    torchvision.transforms.RandomHorizontalFlip(p=.55),
    torchvision.transforms.Normalize(mean= [0.4760, 0.4517, 0.3907], 
                                     std= [0.2362, 0.2314, 0.2307])

])

dataset = ImageDataset(root_directory=path, transform=transform)

In [3]:
#split data
train_size, val_size = int(dataset.__len__() * .8) , int(dataset.__len__() * .2)

ran_gen= torch.Generator().manual_seed(24)

train_data, val_data = torch.utils.data.random_split(dataset, 
                                                    [train_size, val_size],
                                                     generator= ran_gen)




In [4]:
#create dataloaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size=64, shuffle=True)

In [5]:
# #calculcate train data stats
# from src.modeling import calculate_channel_means, calculate_channel_sds

# display(calculate_channel_means(train_data, 'image'))
# #tensor([0.4760, 0.4517, 0.3907])

# display(calculate_channel_sds(train_data, 'image'))
# #tensor([0.2362, 0.2314, 0.2307])


## Model Evaluation
We will evaluate performances using EfficientNet, Resnet18, and VGG16

In [6]:
#init device, criterion, training loop, num_classes
from src.modeling import train_model, model_check_cuda, plot_loss, plot_acc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device is:", device )

criterion = torch.nn.CrossEntropyLoss()

num_classses = len(dataset.class_labels)

device is: cuda


### EfficientNet Model Performance

In [7]:
#init efficientNet and move to Device
efficientNet = torchvision.models.efficientnet_b0()
print(efficientNet.classifier[-1])

efficientNet.classifier[-1] = torch.nn.Linear(in_features= 1280, out_features = num_classses)

efficientNet.to(device)
model_check_cuda(efficientNet)

Linear(in_features=1280, out_features=1000, bias=True)


In [8]:
#init optimizer and train hyperparams
opt_params = {#"params": None,
              "lr":.001,
              "momentum": .9}

train_params = {#'model': None,
                'dataloader':train_dataloader, 
                'epochs':10,
                #'optimizer': None,
                'criterion':criterion, 
                'device':device}

In [9]:
#Train efficientNet
import torch.optim as optim

opt_effNet = optim.SGD(params= efficientNet.parameters(), **opt_params)

efficientNet_dict = {}

efficientNet_dict = train_model(model= efficientNet,
                                optimizer= opt_effNet,
                                **train_params)


/home/kishen/.virtualenvs/stanford_dogs/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Statistics:
epoch loss:  4.799
epoch acc: 1.06%

Epoch 2 Statistics:
epoch loss:  4.781
epoch acc: 1.66%

Epoch 3 Statistics:
epoch loss:  4.742
epoch acc: 1.94%

Epoch 4 Statistics:
epoch loss:  4.651
epoch acc: 2.59%

Epoch 5 Statistics:
epoch loss:  4.55
epoch acc: 3.22%

Epoch 6 Statistics:
epoch loss:  4.446
epoch acc: 3.89%

Epoch 7 Statistics:
epoch loss:  4.34
epoch acc: 4.77%

Epoch 8 Statistics:
epoch loss:  4.244
epoch acc: 5.95%

Epoch 9 Statistics:
epoch loss:  4.157
epoch acc: 6.43%

Epoch 10 Statistics:
epoch loss:  4.068
epoch acc: 7.26%



In [10]:
#plot_acc(efficientNet_dict) 

### Resnet Model Performance

In [12]:
resnet18 = torchvision.models.resnet18(weights='IMAGENET1K_V1')
num_ftrs = resnet18.fc.in_features

resnet18.fc = torch.nn.Linear(num_ftrs, num_classses)
resnet18.to(device)
model_check_cuda(resnet18)


In [14]:
opt_resnet18 = optim.SGD(resnet18.parameters(), 
                          **opt_params)
                
resnet18_dict = {}

resnet18_dict = train_model(model=resnet18, 
                         optimizer = opt_resnet18,
                         **train_params)

/home/kishen/.virtualenvs/stanford_dogs/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch 1 Statistics:
epoch loss:  3.674
epoch acc: 27.47%

Epoch 2 Statistics:
epoch loss:  1.907
epoch acc: 63.75%

Epoch 3 Statistics:
epoch loss:  1.313
epoch acc: 72.58%

Epoch 4 Statistics:
epoch loss:  1.041
epoch acc: 76.49%

Epoch 5 Statistics:
epoch loss:  0.871
epoch acc: 79.91%

Epoch 6 Statistics:
epoch loss:  0.751
epoch acc: 82.72%

Epoch 7 Statistics:
epoch loss:  0.654
epoch acc: 84.91%

Epoch 8 Statistics:
epoch loss:  0.572
epoch acc: 86.80%

Epoch 9 Statistics:
epoch loss:  0.505
epoch acc: 88.99%

Epoch 10 Statistics:
epoch loss:  0.451
epoch acc: 90.14%



### VGG16 Performance

In [23]:
vgg16 = torchvision.models.vgg16()

vgg16.classifier[-1] = torch.nn.Linear(in_features= 4096,
                                       out_features = num_classses)
vgg16.to(device)




VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
https://www.reddit.com/r/deeplearning/comments/c4vd4b/cant_fine_tune_vgg16_to_classify_dog_breeds/

In [24]:
opt_vgg16 = optim.SGD(vgg16.parameters(),
                      lr= .01,
                      momentum = .9)

vgg16_dict = {}
train_params
vgg16_dict = train_model(model=vgg16, 
                         optimizer = opt_vgg16,
                         **train_params)

Epoch 1 Statistics:
epoch loss:  4.787
epoch acc: 1.06%

Epoch 2 Statistics:
epoch loss:  4.785
epoch acc: 1.11%



KeyboardInterrupt: 